In [4]:
import json
import numpy as np
from pysr import PySRRegressor
import pickle as pkl

from dataset import NormalizedProfilesDataset
from utils import load_normalization_metadata
from models import RNN_New, BasicRNN
from torch.utils.data import DataLoader
import torch

In [5]:
# Paths
model_params_path = "Data/Model/model_parameters.json"
model_save_path = "Data/Model/best_model.pth"
data_folder = "Data/Normalized_Profiles"
save_path = "Figures"

# Load the model parameters
with open(model_params_path, 'r') as f:
    model_params = json.load(f)

# Display the model parameters
print("Loaded model parameters:")
print(json.dumps(model_params, indent=4))

Loaded model parameters:
{
    "model_type": "BasicRNN",
    "RNN_type": "LSTM",
    "nx": 4,
    "ny": 1,
    "nx_sfc": 0,
    "nneur": [
        32,
        32
    ],
    "outputs_one_longer": false,
    "concat": false,
    "batch_size": 4,
    "learning_rate": 0.0001,
    "epochs": 500,
    "input_variables": [
        "pressure",
        "temperature",
        "Tstar",
        "flux_surface_down"
    ],
    "target_variables": [
        "net_flux"
    ]
}


In [6]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Retrieve model_type
model_type = model_params.get("model_type", "RNN_New")

# Initialize the model dynamically based on model_type
if model_type == 'BasicRNN':
    model = BasicRNN(
        RNN_type=model_params['RNN_type'],
        nx=model_params['nx'],
        ny=model_params['ny'],
        nneur=tuple(model_params['nneur']),
        outputs_one_longer=model_params['outputs_one_longer'],
        concat=model_params['concat']
    )
elif model_type == 'RNN_New':
    model = RNN_New(
        RNN_type=model_params['RNN_type'],
        nx=model_params['nx'],
        ny=model_params['ny'],
        nneur=tuple(model_params['nneur']),
        outputs_one_longer=model_params['outputs_one_longer'],
        concat=model_params['concat']
    )
else:
    raise ValueError(f"Unknown model type: {model_type}")

# Load the trained model weights
model.load_state_dict(torch.load(model_save_path, map_location=device))
model.to(device)
model.eval()

print(f"Model '{model_type}' loaded and ready for evaluation.")


/var/folders/8_/lltwgz4x48ncx3q3k86fr_tm0000gq/T/ipykernel_80070/1995295661.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_

Model 'BasicRNN' loaded and ready for evaluation.


In [7]:
# Load normalization metadata
normalization_metadata = load_normalization_metadata()

# Expected length of profiles
expected_length = 50

# Initialize the test dataset
test_dataset = NormalizedProfilesDataset(
    data_folder,
    expected_length=50,
    input_variables=['pressure', 'temperature', 'Tstar', 'flux_surface_down'],
    target_variables=['net_flux']
)

# Create DataLoader
test_loader = DataLoader(test_dataset,batch_size=1,shuffle=False)

print("Test dataset loaded.")


Test dataset loaded.


In [14]:
import torch
import numpy as np
import pickle as pkl
from torch.utils.data import DataLoader
from pysr import PySRRegressor

# Ensure model is in evaluation mode
model.eval()

# We'll gather inputs and outputs from the model
X_all = []
Y_all = []
Preds_all = []

# Iterate over a portion of the test data to build a dataset for symbolic regression
# For symbolic regression, a few thousand samples might be enough.
max_samples = 2000
count = 0

with torch.no_grad():
    for X_batch, Y_batch in test_loader:
        X_batch = X_batch.to(device)  # Move to GPU if available
        Y_batch = Y_batch.to(device)

        # Get model predictions
        Y_pred = model(X_batch)

        # Move data back to CPU and convert to numpy
        X_np = X_batch.cpu().numpy()
        Y_np = Y_batch.cpu().numpy()
        Y_pred_np = Y_pred.cpu().numpy()

        X_all.append(X_np)
        Y_all.append(Y_np)
        Preds_all.append(Y_pred_np)

        count += X_np.shape[0]
        if count >= max_samples:
            break

# Concatenate all arrays
X_all = np.concatenate(X_all, axis=0)  # shape: (N, seq_len, nx)
Y_all = np.concatenate(Y_all, axis=0)  # shape: (N, ny)
Preds_all = np.concatenate(Preds_all, axis=0)  # shape: (N, ny)

# If your model output is multi-dimensional (ny > 1), select one dimension for demonstration
if Preds_all.ndim > 1 and Preds_all.shape[1] > 1:
    Preds_all = Preds_all[:, 0]
    Y_all = Y_all[:, 0]

# Now we have:
# X_all: shape (N, seq_len, nx)
# Preds_all: shape (N,)
# We need a suitable input representation for PySR.
# PySR typically expects a 2D array of inputs (N, features).
# One common approach:
# Flatten sequence input or extract features from it.
# For demonstration, let's just flatten the sequence dimension:
N, seq_len, nx = X_all.shape
X_for_pysr = X_all.reshape(N, seq_len * nx)

Y_for_pysr = Preds_all  # We'll try to fit PySR to match model predictions.

# Optionally, choose a random subset for PySR
subset_size = 1000
rstate = np.random.RandomState(0)
idx = rstate.choice(N, size=subset_size, replace=False)
X_for_pysr = X_for_pysr[idx]
Y_for_pysr = Y_for_pysr[idx]

# Save the recorded data if desired
with open("rnn_recordings.pkl", "wb") as f:
    pkl.dump({"X_for_pysr": X_for_pysr, "Y_for_pysr": Y_for_pysr}, f)

# Perform symbolic regression with PySR
model_pysr = PySRRegressor(
    niterations=50,
    binary_operators=["+", "-", "*", "/"],
    unary_operators=["cos", "sin", "exp", "log", "square"],
    progress=True,
    random_state=0,
    deterministic=True,
    parallelism='serial'
)
model_pysr.fit(X_for_pysr, Y_for_pysr)

print("Discovered equations:")
print(model_pysr.equations_)

best_equation = model_pysr.get_best()
print("Best equation found by PySR:")
print(best_equation)


/opt/anaconda3/lib/python3.12/site-packages/pysr/sr.py:2737: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/pysr/sr.py:2222: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, and consider using a larger `maxsize`.
  warnings.warn(
[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:


Discovered equations:
    complexity      loss                                           equation  \
0            1  0.995036                                       -0.002542921   
1            2  0.980967                                           cos(x75)   
2            3  0.000718                                  x87 * -0.99715626   
3            5  0.000700                          (-1.1441721 * x31) / x189   
4            6  0.000695                      (exp(x7) * 0.0014810576) - x7   
5            7  0.000446          (exp(square(x163)) * 7.4602765e-5) - x143   
6            8  0.000446       sin(exp(square(x107)) * 7.4602765e-5) - x163   
7            9  0.000418  (exp(square(x115)) * 7.84769e-5) - (x7 + 0.005...   
8           11  0.000397  (exp(square(x167)) * 8.751666e-5) - ((x195 + 0...   
9           12  0.000396  (square(square(exp(x11) * 0.040987138)) + (-0....   
10          14  0.000390  (8.751666e-5 * exp(square(x27))) - ((0.0056887...   
11          16  0.000383  squa

UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 4094-4095: unexpected end of data

Error in callback _flush_stdio (for post_execute):


UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 4094-4095: unexpected end of data

Error in callback _flush_stdio (for post_execute):


UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 4094-4095: unexpected end of data

Error in callback _flush_stdio (for post_execute):


UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 4094-4095: unexpected end of data